In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

# Historical target table
Hist_table=HTable.read(1409, with_units=False)
Hist_table


hours,Entries
IntegerAndSexagesimal,Historical
0 ;,"00 ; 01,32"
0 ;,"00 ; 03,05"
0 ;,"00 ; 04,37"
0 ;,"00 ; 06,10"
0 ;,"00 ; 07,42"
0 ;,"00 ; 09,15"
0 ;,"00 ; 10,47"
0 ;,"00 ; 12,20"
0 ;,"00 ; 13,52"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(30):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)


In [4]:
# mean motion parameter from the Parisian Alfonsine tables
daily_mean=Historical(Sexagesimal("0;36,59,27,23,59,31"),7)
REVO=Historical(Sexagesimal("6,0;0,0"),7)

for i in range(30):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=3):
        Test_recomp[i][1]=daily_mean*((i+1)/24)%REVO
        with set_precision(tmode=TruncatureMode.TRUNC, pmode=2):
            Test_recomp[i][1]=1*Test_recomp[i][1]
            Delta_recomp[i][1]=3600*(Hist_table[i][1]-Test_recomp[i][1])

In [5]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(-1.5, nan)

In [6]:
B.astype(float)

,Entries
hours,
0 ;,0.0
0 ;,1.0
0 ;,0.0
0 ;,1.0
0 ;,0.0
0 ;,1.0
0 ;,0.0
0 ;,1.0
0 ;,0.0


In [7]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Venus mean hours.csv')
B.to_csv('Venus mean hours.zip', index=False, compression=compression_opts)